# 如何向链的状态中添加值
:::info 前提条件
本指南假设您熟悉以下概念：- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)- [链式运行](/docs/how_to/sequence/)- [并行调用可运行对象](/docs/how_to/parallel/)- [自定义函数](/docs/how_to/functions/)- [数据透传](/docs/how_to/passthrough)
:::
[在链的步骤间传递数据](/docs/how_to/passthrough)的另一种方式是：保持链状态的当前值不变，同时在指定键下分配新值。[`RunnablePassthrough.assign()`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html#langchain_core.runnables.passthrough.RunnablePassthrough.assign)静态方法接收输入值，并将额外参数传递给assign函数。
这在常见的 [LangChain 表达式语言](/docs/concepts/lcel) 模式中非常有用，该模式通过逐步添加字典内容来构建输入，以供后续步骤使用。
这是一个示例：

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'extra': {'num': 1, 'mult': 3}, 'modified': 2}

让我们来分解一下这里发生了什么。
- 链的输入是 `{"num": 1}`。该输入会被传递至一个 `RunnableParallel` 中，该组件会并行调用所接收到的可运行对象并共享此输入。- 调用 `extra` 键下的值。`RunnablePassthrough.assign()` 保留了输入字典中的原始键（`{"num": 1}`），并分配了一个名为 `mult` 的新键。其值为 `lambda x: x["num"] * 3)`，即 `3`。因此，结果为 `{"num": 1, "mult": 3}`。- `{"num": 1, "mult": 3}` 被返回给 `RunnableParallel` 调用，并设置为键 `extra` 的值。- 与此同时，`modified` 键被调用。结果为 `2`，因为 lambda 函数从其输入中提取了一个名为 `"num"` 的键并加一。
因此，结果为 `{'extra': {'num': 1, 'mult': 3}, 'modified': 2}`。
## 流式传输
该方法的一个便利特性是允许值在可用时立即传递。为了展示这一点，我们将使用`RunnablePassthrough.assign()`在检索链中直接返回源文档：

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

generation_chain = prompt | model | StrOutputParser()

retrieval_chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
} | RunnablePassthrough.assign(output=generation_chain)

stream = retrieval_chain.stream("where did harrison work?")

for chunk in stream:
    print(chunk)

{'question': 'where did harrison work?'}
{'context': [Document(page_content='harrison worked at kensho')]}
{'output': ''}
{'output': 'H'}
{'output': 'arrison'}
{'output': ' worked'}
{'output': ' at'}
{'output': ' Kens'}
{'output': 'ho'}
{'output': '.'}
{'output': ''}


我们可以看到，第一个数据块包含原始的`"question"`，因为这部分内容立即可用。第二个数据块包含`"context"`，因为检索器是第二个完成的。最后，来自`generation_chain`的输出会以数据块形式实时流式传输，一旦生成就立即推送。
## 后续步骤
现在你已经学会了如何通过链传递数据，以帮助格式化流经链的数据。
要了解更多信息，请参阅本节中关于可运行项的其他操作指南。